In [1]:
# importing libraries for the API connection

import tweepy
from tweepy.auth import OAuthHandler
import pandas as pd

In [2]:
# generating a dict with the keys for the API connection

secrets_dict={}
secrets_file = open('tweepy-keys.txt')
for line in secrets_file:
  (key,value) = line.split(':')
  secrets_dict[key] = value[:-1]

In [3]:
# creating the API cursor

auth = tweepy.OAuthHandler(secrets_dict['API Key'], secrets_dict['API secret'])
auth.set_access_token(secrets_dict['Access token'], secrets_dict['Access secret'])
api = tweepy.API(auth)

In [4]:
def scrape(words, date_since, numtweet):

# Creating DataFrame using pandas
  db = pd.DataFrame(columns=['username', 'description', 'location', 'following',
              'followers', 'totaltweets', 'retweetcount', 'text', 'hashtags'])

# We are using .Cursor() to search through twitter for the required tweets.
# The number of tweets can be restricted using .items(number of tweets)
  tweets = tweepy.Cursor(api.search, q=words, lang="en",
            since=date_since, tweet_mode='extended').items(numtweet)

# .Cursor() returns an iterable object. Each item in
# the iterator has various attributes that you can access to
# get information about each tweet
  list_tweets = [tweet for tweet in tweets]

# Counter to maintain Tweet Count
  i = 1

# we will iterate over each tweet in the list for extracting information about each tweet
  for tweet in list_tweets:
    username = tweet.user.screen_name
    description = tweet.user.description
    location = tweet.user.location
    following = tweet.user.friends_count
    followers = tweet.user.followers_count
    totaltweets = tweet.user.statuses_count
    retweetcount = tweet.retweet_count
    hashtags = tweet.entities['hashtags']
  
# Retweets can be distinguished by a retweeted_status attribute,
# in case it is an invalid reference, except block will be executed
    try:
      text = tweet.retweeted_status.full_text
    except AttributeError:
      text = tweet.full_text
    hashtext = list()
    for j in range(0, len(hashtags)):
      hashtext.append(hashtags[j]['text'])

    # Here we are appending all the extracted information in the DataFrame
    ith_tweet = [username, description, location, following,
          followers, totaltweets, retweetcount, text, hashtext]
    db.loc[len(db)] = ith_tweet

  

# we will save our database as a CSV file.
  return db

In [21]:
# define forloop that scrapes the necessary tweets and concatenates them

import time # twitter limitates the amount of scrapping you can do, so we need to do timesleep
from datetime import date

# politic is the list of politicians to scrape for
# Enter Date since The Tweets are required in yyyy-mm-dd
# The minutes its the time it will spend between each iteration of the scrape
# numtweet is the number of tweets for each hashtag
# file is the older file that i might have and i want to concatenate with

# WARNING: Remove the default politicians before deploying

def scrapping_engine(date_,politic=['#JoeBiden','#BernieSanders','#MikePence','#TedCruz'], minutes=15, numtweet=500, file=0):

    # we need to create a dataframe to deposit the tweets that we scrape

    columns = ['target','Unnamed: 0', 'username', 'description', 'location', 'following',
           'followers', 'totaltweets', 'retweetcount', 'text', 'hashtags'] # this are the columns that for the dataframe

    politic_df = pd.DataFrame(dict(), columns=columns) # creation of the dataframe

    # this is the loop for the scrapping of every politician

    for element in politic:
        subdata = scrape(element, date_, numtweet)
        subdata['target'] = element
        politic_df = politic_df.append(subdata)
        time.sleep(60 * minutes) # need to take it easy!
        
    csv = file
    
    politic_df = appending(politic_df, csv)
    
    spams_count = dict(politic_df['text'].value_counts())

    spams = []

    for x,y in spams_count.items():
        if y > 1:
            spams.append(x)

    counter = 0

    politic_df = politic_df.reset_index()

    for element in politic_df['text']:
        if element in spams:
            politic_df.drop(index=counter, inplace=True)
        counter += 1
    
    new_date = 'scraped_from_' + date_ + '_to_' + str(date.today())
    
    if csv != 0:
        politic_df.to_csv(new_date + '.csv', index_label=False)
        return politic_df
    else:
        politic_df.to_csv(new_date + '.csv', index_label=False)
        return politic_df


In [6]:
# define a function that adds the newly scraped file to the rest

def appending(db, file):
    older = pd.read_csv(file)
    return db.append(older) # returns the new database appended

In [7]:
def politicians_input():
    print('Enter the politicians names (separated by comma, E.g: ´Joe Biden, Bernie Sanders´): ')
    string_ = input()
    string = string_.replace(' ','')
    list_strings = string.split(',')

    return [('#' + x) for x in list_strings]

In [22]:
to_scrape = politicians_input()

# do a function for date input

politic_df = scrapping_engine('2021-07-22', to_scrape, 0.1, 10, 'data.csv')

Enter the politicians names (separated by comma, E.g: ´Joe Biden, Bernie Sanders´): 
Joe Biden, Bernie Sanders, Ted Cruz, Mike Pence


In [23]:
politic_df.shape

(1049, 13)

In [24]:
politic_df.head()

,index,target,Unnamed: 0,username,description,location,following,followers,totaltweets,retweetcount,text,hashtags,Unnamed: 0.1
0,0,#JoeBiden,NaN,Michell81478626,"I live in Minnesota, I love my cat, dog and bi...","Minnesota, USA",69,12,60,2,#Congress #chuckSchumer #nancyPelosi #democrat...,"[Congress, chuckSchumer, nancyPelosi, democrat...",NaN
2,2,#JoeBiden,NaN,DerekOsheaShow,Derek O'Shea Show Politically Homeless Daily C...,Everywhere you want to be,3435,781,5095,0,Who is Defunding Police | Conspiracy Theories ...,"[Covid19News, JoeBiden, BreakingNews, politica...",NaN
3,3,#JoeBiden,NaN,Tim20026046,"White-haired, upper fifties, libertarian dad, ...",,485,17,4239,98,"#JoeBiden Tells The World Another Whopper ""You...",[JoeBiden],NaN
4,4,#JoeBiden,NaN,lilgoddamn_III,#bitcoin\n§LilGoddamn,The Moon 🌕,900,297,28972,57,Democrats Future Agenda...\n\n#NacyPelosi :\nL...,"[NacyPelosi, JoeBiden]",NaN
5,5,#JoeBiden,NaN,LisaMaret,"Founder, Tea Party WDC ""Return 2 commerce, cha...",Alexandria Va,1446,1735,69535,0,"@taxreformer Big surprise, the IRS doesn't aud...","[LarcenousClass, Democrats, JoeBiden, Progress...",NaN


In [25]:
# now we need to clean the tweets
# importing the necessaire packages


import re
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
nltk.download('wordnet')
from nltk.corpus import wordnet

nltk.download('averaged_perceptron_tagger')

from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [26]:
# the main cleaning step

def clean_up(s):
    element1 = re.sub('(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)', '', s) # remove links
    element2 = re.sub('[^a-zA-Z0-9]', ' ', element1) # remove non character symbols
    element3 = re.sub('amp', '', element2) # twitter has &amp as a special character
    return (re.sub('\d+',' ',element3)).lower() # remove any digits and lowercase everything

In [27]:
# tokenize the text

def tokenize(s):
    return word_tokenize(s)

In [28]:
# categorize function to help the next function that is lemmatize

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper() # gets first letter of POS categorization
    tag_dict = {"J": wordnet.ADJ, 
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN) # get returns second argument if first key does not exist

In [29]:
# lemmatize to reduce the word to it most radical form

def lemmatize(l):
  
    lem = WordNetLemmatizer()
    lemmatized = [lem.lemmatize(w,get_wordnet_pos(w)) for w in l]
    
    return lemmatized

In [30]:
# remove the stop words

def remove_stopwords(l):
    
    filtered_sentence = []
    
    for w in l:
        if w not in stopwords.words('english'):
            filtered_sentence.append(w)
    
    return filtered_sentence

In [31]:
# function that combines all of the cleaning functions and does everything

def cleaning_engine(df,column='text'):
    df['text_processed'] = df[column].apply(clean_up)
    df['text_processed'] = df['text_processed'].apply(tokenize)
    df['text_processed'] = df['text_processed'].apply(lemmatize)
    return df['text_processed'].apply(remove_stopwords)
    

In [32]:
politic_df['text_processed'] = cleaning_engine(politic_df)

In [33]:
# think about blobbing the results

from nltk.probability import FreqDist
import itertools

def get_most_common_words(amount, column=politic_df['text_processed']):

    blob = column.tolist()

    blob = itertools.chain.from_iterable(blob)

    fdist = FreqDist(blob)

    common = dict(fdist.most_common(amount)) # review this number

    return list(common.keys())

In [34]:
most_common = get_most_common_words(1000)

In [35]:
most_common

['joebiden',
 'tedcruz',
 'berniesanders',
 'biden',
 'mikepence',
 'trump',
 'cnn',
 'gop',
 'texas',
 'foxnews',
 'get',
 'covid',
 'democrat',
 'aoc',
 'maga',
 'say',
 'rondesantis',
 'gregabbott',
 'variant',
 'u',
 'delta',
 'texan',
 'billlee',
 'mikeparson',
 'dougducey',
 'kristinoem',
 'kevinmccarthy',
 'penny',
 'msnbc',
 'usa',
 'want',
 'laurenboebert',
 'like',
 'free',
 'bernie',
 'go',
 'joe',
 'president',
 'lindseygraham',
 'would',
 'marcorubio',
 'potus',
 'morningjoe',
 'abc',
 'people',
 'mitchmcconnell',
 'work',
 'state',
 'jan',
 'cuba',
 'blm',
 'need',
 'mattgaetz',
 'texit',
 'order',
 'best',
 'party',
 'pelosi',
 'america',
 'tatereeves',
 'secede',
 'cruz',
 'know',
 'think',
 'make',
 'nytimes',
 'blacklivesmatter',
 'harris',
 'mike',
 'cbs',
 'news',
 'manchin',
 'bitcoin',
 'nyc',
 'lie',
 'nothing',
 'one',
 'american',
 'take',
 'kingjames',
 'ted',
 'shipping',
 'texasforever',
 'republicoftexas',
 'texassecede',
 'secession',
 'gregabott',
 'right

In [ ]:
# add anything that was not included in that list

#spams.append('good morning happy friday mikepence even trust secret service protect jan th fear coup maga gallows ready still call donald trump personal friend mary trump need write book abt psycho mike penny morningjoe penny co rz sbyy')

In [33]:
# now can start developing the features for our model

from sklearn.feature_extraction.text import CountVectorizer

# first i will vectorize all of the words in each tweet by the most common words

bow_vect = CountVectorizer(vocabulary=most_common, max_features=1000)

# fit creates one entry for each different word seen

X = bow_vect.fit_transform(politic_df['blobbed']).toarray()

In [ ]:
# we want to standardize the features for improving the model

from sklearn.preprocessing import StandardScaler

# create object

scaler = StandardScaler()

# fit

scaler.fit(X)

# transform 

X_scaled = scaler.transform(as_df)